In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

from selenium import webdriver

options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")  # sandbox 주요 요소 처리불가-> nosandbox 처리 가능

#headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'}
#options.add_argument('headers')
driver = webdriver.Chrome(options=options)

In [3]:

url='https://huggingface.co/papers/2312.07409'
driver.get(url)

all_video=[]
all_video_url=[]


if driver.find_elements(By.TAG_NAME,'video'): # 동영상
    all_video.append('yes')
    image=driver.find_element(By.XPATH,'/html/body/div/main/div/section[1]/div/video')
    img_srtc=image.get_attribute('src')
    all_video_url.append(img_srtc)
    print(all_video_url)
else:
    all_video.append('no')

['https://cdn-uploads.huggingface.co/production/uploads/60f1abe7544c2adfd699860c/pPBftW9u_SvGH1HLzFDu-.mp4']


In [11]:
url='https://huggingface.co/papers?date=2023-12-13'
driver.get(url)
time.sleep(1)

# 전 날 페이지로 넘기기
# for d in range(2):

all_title=[]
all_news=[]
all_url=[]
all_author=[] 
all_stars=[]
all_days=[]
word=[]
all_images=[]
all_images_url=[]
all_video=[]
all_video_url=[]

titles=driver.find_elements(By.TAG_NAME,'h3')# 제목
stars=driver.find_elements(By.TAG_NAME,'a>div>div') # 좋아요
day=driver.find_elements(By.TAG_NAME,'time')

for i in titles:
    all_title.append(i.text)
all_title=[i for i in all_title if i]  # 빈칸제거

for i in stars:
    all_stars.append(i.text)

for i in day:
    all_days.append(i.text)

print(all_days)
    
word=all_days[0]
word=word.replace('\n','')
all_days2=[]
all_days2.append(word)


for n in all_title:

    print(n)
    driver.find_element(By.PARTIAL_LINK_TEXT,n).click() #페이지 들어가기
    time.sleep(1)
    
    nourl=driver.current_url#url 저장
    all_url.append(nourl)
    time.sleep(1)
    
    news=driver.find_element(By.TAG_NAME,'p') # 뉴스 본문 저장
    all_news.append(news.text)


    author=driver.find_element(By.CLASS_NAME,'whitespace-nowrap.underline') #저자 
    all_author.append(author.text)


    # if driver.find_elements(By.TAG_NAME,'img'): #이미지 paper 에는 이미지 없음
    #     all_images.append('yes')
    #     image=driver.find_element(By.XPATH,'/html/body/div/main/div[2]/section[1]/div[3]/div[2]/p[4]/a/img')
    #     img_srtc=image.get_attribute('src')
    #     all_images_url.append(img_srtc)
    # else:
    #     all_images.append('yes')
        

    if driver.find_elements(By.TAG_NAME,'video'): # 동영상
        all_video.append('yes')
        image=driver.find_element(By.XPATH,'/html/body/div/main/div/section[1]/div/video')
        img_srtc=image.get_attribute('src')
        all_video_url.append(img_srtc)
    else:
        all_video.append('no')
        all_video_url.append('-')

    driver.back()
    time.sleep(1)

    # # 잠깐 쉬고 전 페이지 가기
    # time.sleep(3)
    # driver.find_element(By.CLASS_NAME,'btn.gap-2').click() # 사이트 전 날

all_days2=all_days2*len(all_title) # 날짜 개수 제목개수만큼 늘리기


sumnews=[]
for t,n,u,a,s,d,v,vr in zip(all_title,all_news,all_url,all_author,all_stars,all_days2,all_video,all_video_url):
    singles={'title':t,'news':n,'url':u,'author':a,'like':s,'day':d,'video':v,'video_url':vr}
    sumnews.append(singles)

import pandas as pd
news_df=pd.DataFrame(sumnews)
print(news_df)

import os
if not os.path.exists('output.csv'):
    news_df.to_csv('output.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    news_df.to_csv('output.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

['DEC\n13']
FreeInit: Bridging Initialization Gap in Video Diffusion Models
FreeControl: Training-Free Spatial Control of Any Text-to-Image Diffusion Model with Any Condition
DiffMorpher: Unleashing the Capability of Diffusion Models for Image Morphing
VILA: On Pre-training for Visual Language Models
Alignment for Honesty
CCM: Adding Conditional Controls to Text-to-Image Consistency Models
Steering Llama 2 via Contrastive Activation Addition
Interfacing Foundation Models' Embeddings
COLMAP-Free 3D Gaussian Splatting
Rethinking Compression: Reduced Order Modelling of Latent Features in Large Language Models
Honeybee: Locality-enhanced Projector for Multimodal LLM
How Well Does GPT-4V(ision) Adapt to Distribution Shifts? A Preliminary Investigation
Fast Training of Diffusion Transformer with Extreme Masking for 3D Point Clouds Generation
Llama Guard: LLM-based Input-Output Safeguard for Human-AI Conversations
PEEKABOO: Interactive Video Generation via Masked-Diffusion
"I Want It That Way

In [12]:
news_df

,title,news,url,author,like,day,video,video_url
0,FreeInit: Bridging Initialization Gap in Video...,Though diffusion-based video generation has wi...,https://huggingface.co/papers/2312.07537,Tianxing Wu,21,DEC13,no,-
1,FreeControl: Training-Free Spatial Control of ...,Recent approaches such as ControlNet offer use...,https://huggingface.co/papers/2312.07536,Sicheng Mo,12,DEC13,no,-
2,DiffMorpher: Unleashing the Capability of Diff...,Diffusion models have achieved remarkable imag...,https://huggingface.co/papers/2312.07409,Kaiwen Zhang,11,DEC13,yes,https://cdn-uploads.huggingface.co/production/...
3,VILA: On Pre-training for Visual Language Models,Visual language models (VLMs) rapidly progress...,https://huggingface.co/papers/2312.07533,Ji Lin,11,DEC13,no,-
4,Alignment for Honesty,Recent research has made significant strides i...,https://huggingface.co/papers/2312.07000,Yuqing Yang,9,DEC13,no,-
5,CCM: Adding Conditional Controls to Text-to-Im...,Consistency Models (CMs) have showed a promise...,https://huggingface.co/papers/2312.06971,Jie Xiao,8,DEC13,no,-
6,Steering Llama 2 via Contrastive Activation Ad...,We introduce Contrastive Activation Addition (...,https://huggingface.co/papers/2312.06681,Nina Rimsky,7,DEC13,no,-
7,Interfacing Foundation Models' Embeddings,"We present FIND, a generalized interface for a...",https://huggingface.co/papers/2312.07532,Xueyan Zou,6,DEC13,no,-
8,COLMAP-Free 3D Gaussian Splatting,While neural rendering has led to impressive a...,https://huggingface.co/papers/2312.07504,Yang Fu,6,DEC13,no,-
9,Rethinking Compression: Reduced Order Modellin...,Due to the substantial scale of Large Language...,https://huggingface.co/papers/2312.07046,Arnav Chavan,6,DEC13,no,-


In [110]:
url='https://huggingface.co/papers/2312.03461'
driver.get(url)
all_days=[]
# day=driver.find_elements(By.CLASS_NAME,'relative.flex.flex-col.items-stretch')
day=driver.find_elements(By.TAG_NAME,'time')

for i in day:
    all_days.append(i.text)

word=all_days[0]
word=word.replace('\n','')
all_days=[]
all_days.append(word)

all_days=all_days*len(all_title)
print(all_days)




all_video=[]
all_video_url=[]
if driver.find_elements(By.TAG_NAME,'video'):
    all_video.append('yes')
    print(all_video)
    videourl=driver.find_elements(By.XPATH,'/html/body/div/main/div/section[1]/div/video')
    print(videourl)

    for i in videourl:
        asd=i.get_attribute('video')
        print(asd)

else:
    all_video.append('no')


['6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago', '6 days ago']
['yes']
[<selenium.webdriver.remote.webelement.WebElement (session="e85c160ef56cfb2453095867ed93d43d", element="48F15FD037DD7B17FC92181CE2F0D398_element_344")>]
None
